In [5]:
# Cell 0: Paths & imports
from pathlib import Path
import os, math, json, random, statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image, ImageOps, ImageStat, ImageFilter
import cv2  
import torch
from torchvision import transforms, models

plt.rcParams['figure.dpi'] = 140
sns.set_theme(style="whitegrid")


DATA_ROOT = "C:/TRAIN/Deep Learning/vietnamese-foods/Images"   # <== đổi nếu cần
TRAIN_DIR = f"{DATA_ROOT}/Train"
VAL_DIR   = f"{DATA_ROOT}/Validate"





IMG_EXTS  = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# Kiểm tra tồn tại (tùy bộ dữ liệu của bạn có hoặc không có val)
assert TRAIN_DIR.exists(), f"Không thấy thư mục train: {TRAIN_DIR}"
if not VAL_DIR.exists():
    print("⚠️ Không thấy thư mục val, sẽ chỉ quét train.")


AttributeError: 'str' object has no attribute 'exists'

In [4]:
# Cell 1: Scan dataset -> DataFrame
def scan_split(split_dir, split_name):
    rows = []
    for cls_dir in sorted([d for d in split_dir.iterdir() if d.is_dir()]):
        cls = cls_dir.name
        for p in cls_dir.rglob("*"):
            if p.suffix.lower() in IMG_EXTS:
                rows.append({"split": split_name, "cls": cls, "path": str(p)})
    return rows

rows = scan_split(TRAIN_DIR, "train") + scan_split(VAL_DIR, "val")
df = pd.DataFrame(rows)
print(df.head(), "\nTổng ảnh:", len(df), "— Lớp:", df['cls'].nunique())

# Đọc metadata ảnh cơ bản (kích thước, tỷ lệ, diện tích, độ sáng, độ mờ...)
def image_meta(path):
    try:
        with Image.open(path) as im:
            im = im.convert("RGB")
            w, h = im.size
            # sáng & độ bão hòa cơ bản
            stat = ImageStat.Stat(im)
            mean_rgb = [v for v in stat.mean]          # 3 kênh
            brightness = sum(mean_rgb)/3.0
            # độ mờ (variance of laplacian)
            arr = np.array(im)
            gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
            blur = cv2.Laplacian(gray, cv2.CV_64F).var()
            # tỷ lệ
            ar = w / h
            return w, h, ar, w*h, brightness, blur
    except Exception as e:
        return None

meta = df['path'].apply(image_meta)
mask_ok = meta.notna()
df = df[mask_ok].copy()
df[["w","h","aspect","area","brightness","blur_var"]] = pd.DataFrame(meta[mask_ok].tolist(), index=df.index)
df.reset_index(drop=True, inplace=True)
df.head()


AttributeError: 'str' object has no attribute 'iterdir'